# SPARRA data examples

Used the code in the original notebook to generate some static example files

## PIS

Each row is a prescription payment _event_. Each prescription event is characterised by a BNF section (see e.g. https://openprescribing.net/bnf/), a date, and a number of items. Features might be things like:
1. Number of distinct BNF sections for an ID
2. Count of items that come from one of a set of BNF sections
3. Total number of items across all BNF sections


In [28]:
import pandas as pd
pis_data_path = "./example_data/random_pis_data.csv"

In [29]:
pis_data = pd.read_csv(pis_data_path, dtype={'bnf_section': str})
pis_data.head()

,id,paid_date,bnf_section,num_items
0,19,2017-12-15,0106,3
1,19,2016-08-11,0103,5
2,19,2015-07-07,0103,1
3,7,2017-03-14,0106,1
4,3,2015-08-07,0105,1


### Look at patient 0 only

In [30]:
p0 = pis_data[pis_data['id'] == 0]
p0.head(30)

,id,paid_date,bnf_section,num_items
32,0,2017-01-12,0107,4
44,0,2016-11-08,0109,4
57,0,2017-06-27,0107,2
96,0,2015-08-17,0103,4


Patient with ID 0 (above) has **4 total entries** on this list, their bnf_selections are 107, 109, 107, and 103. That is **3 unique** bnf_selections over the four entries. The earliest entry for patient 0 is **2015-08-17** and the latest entry is **2017-06-27**. Over the four total entries, the patient had the following num_items prescribed: 4, 4, 2, and 4 which sums to **14** items prescribed

### Example PIS features

In [24]:
# distinct bnf
n_distinct = pis_data.groupby('id').agg(n_distinct_bnf = pd.NamedAgg(column='bnf_section', aggfunc='nunique')).reset_index()
n_distinct.head()

,id,n_distinct_bnf
0,0,3
1,1,3
2,2,4
3,3,6
4,4,3


In [25]:
# total prescribed items
total_items = pis_data.groupby('id').agg(n_total_items = pd.NamedAgg(column='num_items', aggfunc='sum')).reset_index()
total_items.head()

,id,n_total_items
0,0,14
1,1,10
2,2,14
3,3,27
4,4,11


Looking back at patient 0, for only selections 103 and 107 we get:

In [26]:
p0[p0['bnf_section'].isin(["0103","0107"])].head(10)

,id,paid_date,bnf_section,num_items
32,0,2017-01-12,0107,4
57,0,2017-06-27,0107,2
96,0,2015-08-17,0103,4


There have been 3 events where either bnf_section 103 or 107 was prescibed. The total number of items based on this filtering is **10**

In [27]:
# particular BNF (total prescribed items after some filtering on section)
pis_diar_and_anti = pis_data.loc[
    pis_data['bnf_section'].isin(["0103","0107"]), :
    ].groupby('id').agg(n_diar_and_anti = pd.NamedAgg(column='num_items', aggfunc='sum')).reset_index()
pis_diar_and_anti.head()

,id,n_diar_and_anti
0,0,10
1,1,2
2,2,5
3,3,3
4,4,6


## SMR04 (psych admissions -- example where there are multiple episodes per stay)

Bit harder to generate, so simple simulation

- Each row is an _episode_. Multiple episodes make up a stay.
- The `cis_marker` field can be used to group episodes from one id into a stay. I.e. all rows for a (`id`, `cis_marker`) tuple correspond to a single stay.
- `episode_within_cis` tells us the order of the episodes within a stay
- I've added a random code to show what the features might be like

In [42]:
smr04_data_path = "./example_data/random_smr04_data.csv"
smr04_data = pd.read_csv(smr04_data_path, parse_dates=["admission_date", "discharge_date"])
smr04_data.head(15)

,id,admission_date,discharge_date,cis_marker,episode_within_cis,some_code
0,5,2017-04-07,2017-04-08,29,1,a
1,5,2017-04-08,2017-04-09,29,2,c
2,5,2017-04-09,2017-04-10,29,3,d
3,17,2017-07-12,2017-07-12,26,1,c
4,3,2017-06-08,2017-06-08,54,1,e
5,16,2015-09-27,2015-09-27,88,1,b
6,16,2015-09-27,2015-09-28,88,2,e
7,16,2015-09-28,2015-09-29,88,3,b
8,16,2015-09-29,2015-10-01,88,4,e
9,16,2015-10-01,2015-10-01,88,5,c


For example - patient 16 had a **stay** which spanned from **2015-09-27** to **2015-10-03** and consisted of **7 episodes** (with cis_marker of 88 in that case). The first episode for patient 16 in this stay had a code of **b** and during their 7-episode stay they had codes of **b**, **e**, **c**, and **d**.

In [43]:
p16 = smr04_data[smr04_data["id"]==16]
p16.head(20)

,id,admission_date,discharge_date,cis_marker,episode_within_cis,some_code
5,16,2015-09-27,2015-09-27,88,1,b
6,16,2015-09-27,2015-09-28,88,2,e
7,16,2015-09-28,2015-09-29,88,3,b
8,16,2015-09-29,2015-10-01,88,4,e
9,16,2015-10-01,2015-10-01,88,5,c
10,16,2015-10-01,2015-10-03,88,6,d
11,16,2015-10-03,2015-10-03,88,7,e
81,16,2017-08-23,2017-08-24,89,1,c
82,16,2017-08-24,2017-08-25,89,2,a
83,16,2017-08-25,2017-08-25,89,3,c


Patient 16 also had a 3-episode stay in August 2017, and so on...

In [48]:
p2 = smr04_data[smr04_data["id"]==2]
p2.head(20)

,id,admission_date,discharge_date,cis_marker,episode_within_cis,some_code,length_of_stay
44,2,2016-11-29,2016-11-30,67,1,e,1 days
45,2,2016-11-30,2016-12-02,67,2,b,2 days
46,2,2016-12-02,2016-12-03,67,3,c,1 days
113,2,2015-10-04,2015-10-05,68,1,c,1 days
155,2,2017-03-01,2017-03-02,69,1,d,1 days
156,2,2017-03-02,2017-03-02,69,2,a,0 days


Patient 2 had 3 stays in total. The first stay spanned 2016-11-29 to 2016-12-03 and consisted of 3 episodes, the second stay corresponded to a single-episode stay from 2015-10-04 to 2015-10-05, and the third stay was a 2-episode stay from 2017-03-01 to 2017-03-02. This corresponds to a 4-night, 1-night, and 1-night stay, so 6 in total. Only one of their stays included an episode with code **b**

### Example SMR04 features

Just some examples to hopefully cover the kind of things SPARRA uses.

Note one thing I haven't shown here -- sometimes for episodic data like this you have to be careful when doing _date filtering_. I.e. SPARRA only includes information for _stays_ that start within the relevant time period. Therefore you cannot simply filter on episodes dates, but you need to either:
1. Keep all episodes where the first episode in the stay is within the date range, or
2. Merge into stays first and then filter.

To transform to one row per stay, you'd need to do things like take the admission date from the first episode and the discharge from the last etc.

Note that some SPARRA features count stay-related things based on codes in their first episode (stays are classified as emergency / elective based just on information in their first episode) and some things based on the code appearing in any episode (alcohol related stays are counted as the number of stays where _any_ episode within the stay has an alcohol diagnosis code)

In [45]:
# number of stays per id
# (lots of ways of calculating this) -- this is one example: count the distinct cis_marker
n_stays = smr04_data.groupby('id').agg(n_stays = pd.NamedAgg(column='cis_marker', aggfunc='nunique')).reset_index()
n_stays.head()

,id,n_stays
0,0,5
1,1,5
2,2,3
3,3,8
4,4,3


In [46]:
# total nights in hospital
smr04_data['length_of_stay'] = smr04_data['discharge_date'] - smr04_data['admission_date']
n_bed_days = smr04_data.groupby('id').agg(total_bed_days = pd.NamedAgg(column='length_of_stay', aggfunc='sum')).reset_index()
n_bed_days.head()

,id,total_bed_days
0,0,4 days
1,1,13 days
2,2,6 days
3,3,13 days
4,4,7 days


In [49]:
# number of stays where the first episode had code 'b'
n_stays_b = smr04_data.loc[
    (smr04_data['some_code'] == 'b') & (smr04_data['episode_within_cis'] == 1), :
    ].groupby('id').agg(n_stays_b = pd.NamedAgg(column='cis_marker', aggfunc='nunique')).reset_index()
n_stays_b.head()
# number of bed days for stays where any episode has code 'c'
# ran out of time to implement -- but basically need to keep all _stays_
# where _any_ episode in the stay has a code c and then compute number of bed days

,id,n_stays_b
0,3,3
1,4,1
2,5,1
3,6,1
4,13,2


# SPARRA LTC

This one is a bit different (but will work). The data has one row per ID and one column per Long Term Condition. The values are NA if the person does not have that LTC and the date they were first diagnosed if they do. There are 19 conditions. I've just done 3.

In [19]:
conditions = ['ASTHMA', 'DIABETES', 'PARKINSONS']
data_dict = {'id': [], 'ASTHMA': [], 'DIABETES': [], 'PARKINSONS': []}
start_dates = random_dates(START, END, n=MAX_ID*3, seed=1, replace=True)
date_pos = 0
for i in range(MAX_ID):
    data_dict['id'].append(i)
    for c_pos in range(len(conditions)):
        if np.random.rand() < 0.2:
            # they have this condition
            dat = start_dates[date_pos]
            date_pos += 1
            data_dict[conditions[c_pos]].append(dat)
        else:
            data_dict[conditions[c_pos]].append(np.NaN)
ltc_data = pd.DataFrame(data_dict, index=range(MAX_ID))

In [95]:
ltc_data

,id,ASTHMA,DIABETES,PARKINSONS
0,0,2017-11-27,NaT,NaT
1,1,NaT,2015-08-24,NaT
2,2,NaT,NaT,2018-01-01
3,3,NaT,2017-06-24,NaT
4,4,NaT,NaT,NaT
5,5,NaT,NaT,NaT
6,6,NaT,2016-12-16,NaT
7,7,NaT,NaT,NaT
8,8,NaT,2017-04-27,NaT
9,9,NaT,NaT,NaT


Exmaple features:
1. Years since the LTC diagnosis date (null if no date) - one feature per lTC so there would be three in this case: years_since_asthma, years_since_diabetes etc
2. Number of LTCs (i.e. number of non-null values for each id (row)

## Indicated features
These combine PIS and LTC. E.g. for a particular ID, feature starts with value 0. If they have any prescriptions in a set of BNF sections, then add 1. If they have a particular LTC date, then add 1. So value will be 0, 1, 2. There are five of these -- each with different BNF and LTC combinations.

This is the example where I think we'd want to be able to specify features in the json that combine multiple inputs, and the inputs would be the output of other transformations.

E.g. transform 1: do they have a prescritpion in some BNF section(s) (1 = yes, 0 = no)
transform 2: do they have a date in a particular LTC (1 = yes, 0 = no)
transform 3: transform 1 + transform 2 (per id)

Concrete example, BNF = 0201 and LTC = diabetes. So, need a transformation that computes a "has_0201" feature (0 or 1 depending if theyhave any 0201 PIS rows or now) and a has_diabetes feature (0 or 1 depending if they have null, or a date in LTC)

We would then need a third transformer that would ADD the has_0201 and has_diabetes feature for each id.